# First test of a binary classification of stork nest images
Analog the TensorFlow Tutorial: https://www.tensorflow.org/tutorials/images/classification

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_dir = os.path.join("../images/2019_train", "train")
validation_dir = os.path.join("../images/2019_train", "validation")

In [ ]:
train_stork_dir = os.path.join(train_dir, "stork")  # directory with our training cat pictures
train_no_stork_dir = os.path.join(train_dir, "no_stork")  # directory with our training dog pictures
validation_stork_dir = os.path.join(validation_dir, "stork")  # directory with our validation cat pictures
validation_no_stork_dir = os.path.join(validation_dir, "no_stork")  # directory with our validation dog pictures

In [ ]:
num_stork_tr = len(os.listdir(train_stork_dir))
num_no_stork_tr = len(os.listdir(train_no_stork_dir))

num_stork_val = len(os.listdir(validation_stork_dir))
num_no_stork_val = len(os.listdir(validation_no_stork_dir))

total_train = num_stork_tr + num_no_stork_tr
total_val = num_stork_val + num_no_stork_val

In [ ]:
batch_size = 60
epochs = 15
IMG_HEIGHT = 480
IMG_WIDTH = 640

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode="binary")

In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode="binary")

In [ ]:
sample_training_images, _ = next(train_data_gen)
sample_validation_images, _ = next(val_data_gen)

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_training_images[:5])

In [ ]:
model = Sequential([
    Conv2D(16, 3, padding="same", activation="relu", input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding="same", activation="relu"),
    MaxPooling2D(),
    Conv2D(64, 3, padding="same", activation="relu"),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation="relu"),
    Dense(1)
])

In [ ]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

loss=history.history["loss"]
val_loss=history.history["val_loss"]

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Training Accuracy")
plt.plot(epochs_range, val_acc, label="Validation Accuracy")
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")
plt.show()

In [ ]:
predictions = model.predict(val_data_gen)

In [ ]:
predictions

In [ ]:
predictions.shape

In [ ]:
predictions_prob = tf.round(tf.nn.sigmoid(predictions))
predictions_prob.shape

In [ ]:
fig, axes = plt.subplots(12, 5, figsize=(24, 40))
axes = axes.flatten()
for img, ax, i in zip(sample_validation_images, axes, range(60)):
    ax.imshow(img)
    ax.axis("off")
    ax.set_title(round(predictions.item(i)))
plt.tight_layout()
plt.show()